In [1]:
import sys
from glob import glob
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import unique, join
import pandas as pd
from astropy.time import Time

In [2]:
from source_names_dict import source_names_dict, source_names_w_counterparts, source_names_readable
from get_closest_srcreg import get_src_region_dict
from uvot import read_uvotsource, calc_uvot_flags, filters, filter_colors, filter_markers, load_uvotsource
from xrt import load_xrt, rename_xrt_cols, curves
from lightcurve import LightCurve

In [3]:
def get_first_val(df, col):
    try:
        val = df[col].values[0]
    except IndexError:
        val = np.nan
    except AttributeError:
        val = df[col][0]
    return val

In [4]:
src_region_dict = get_src_region_dict()

Getting source region dict


In [5]:
src_region_dict

{'ESO_243-49_HLX-1': '/mnt/d/anticorr_data/download_scripts/ESO243-49/src.reg',
 'NAME_Holmberg_IX_X-1': '/mnt/d/anticorr_data/download_scripts/Holmberg_IX/src.reg',
 'Holmberg_II_X-1': '/mnt/d/anticorr_data/download_scripts/Holmberg_II/src.reg',
 'M31_ULX-1': '/mnt/d/anticorr_data/download_scripts/M31/src.reg',
 '[LM2005]_NGC_598_ULX1': '/mnt/d/anticorr_data/download_scripts/M33/src.reg',
 'RX_J133001+47137': '/mnt/d/anticorr_data/download_scripts/M51/ULX7_src.reg',
 '[LM2005]_NGC_3031_ULX1': '/mnt/d/anticorr_data/download_scripts/M81/ULX1_src.reg',
 'M82_X-2': '/mnt/d/anticorr_data/download_scripts/M82/src.reg',
 '[LM2005]_NGC_1042_ULX1': '/mnt/d/anticorr_data/download_scripts/NGC1042/src.reg',
 'NAME_NGC_1313_X-1': '/mnt/d/anticorr_data/download_scripts/NGC1313/X-1_src.reg',
 'NAME_NGC_1313_X-2': '/mnt/d/anticorr_data/download_scripts/NGC1313/X-2_src.reg',
 '[LM2005]_NGC_247_ULX1': '/mnt/d/anticorr_data/download_scripts/NGC247/src.reg',
 '[LB2005]_NGC_253_X2': '/mnt/d/anticorr_data/

In [6]:
all_rows = []
for simbad_name, local_name in source_names_dict.items():
    readable_name = source_names_readable[simbad_name]
    try:
        df_xrt_pc   = load_xrt(simbad_name, curve='PC')
        df_xrt_soft = load_xrt(simbad_name, curve='SOFT')
        df_xrt_hard = load_xrt(simbad_name, curve='HARD')
        df_xrt_hr   = load_xrt(simbad_name, curve='HR')
    except:
        row = {}
        row['readable_name'] = readable_name
        all_rows.append(row)
        print(row)
        continue
    
    try:
        print(f'loading uvot {simbad_name}')
        tab_uvot = load_uvotsource(simbad_name, src_region_dict, False)
        print(len(tab_uvot))
        uvot_loaded = True
    except:
        print('Could not load uvot')
        uvot_loaded = False

    for obsid in df_xrt_pc['obsID']:
        sub_pc   = df_xrt_pc[df_xrt_pc['obsID'] == obsid]
        sub_soft = df_xrt_soft[df_xrt_soft['obsID'] == obsid]
        sub_hard = df_xrt_hard[df_xrt_hard['obsID'] == obsid]
        sub_hr   = df_xrt_hr[df_xrt_hr['obsID'] == obsid]
        
        
        row = {}

        mjd           = get_first_val(sub_pc, 'MJD')
        rate_pc       = get_first_val(sub_pc, 'Rate')
        rate_pc_err   = get_first_val(sub_pc, 'Ratepos')
        rate_soft     = get_first_val(sub_soft, 'Rate')
        rate_soft_err = get_first_val(sub_soft, 'Error')
        rate_hard     = get_first_val(sub_hard, 'Rate')
        rate_hard_err = get_first_val(sub_hard, 'Error')
        rate_hr       = get_first_val(sub_hr, 'Rate')
        rate_hr_err   = get_first_val(sub_hr, 'Error')
        

        if uvot_loaded:
            sub_uvot = tab_uvot[tab_uvot['OBSID'] == obsid]
            for filt in filters:
                sub_uvot_filt = sub_uvot[sub_uvot['FILTER'] == filt]
                if len(sub_uvot_filt) > 0:
                    mag              = get_first_val(sub_uvot_filt, 'MAG')
                    mag_err          = get_first_val(sub_uvot_filt, 'MAG_ERR')
                    coi_src_rate     = get_first_val(sub_uvot_filt, 'COI_SRC_RATE')
                    coi_src_rate_err = get_first_val(sub_uvot_filt, 'COI_SRC_RATE_ERR')
                    nsigma           = get_first_val(sub_uvot_filt, 'NSIGMA')
                    
                    row[f'{filt}_MAG']              = f"{mag:.2f} $\pm$ {mag_err:.3f}"
                    row[f'{filt}_COI_SRC_RATE']     = f"{coi_src_rate:.2f} $\pm$ {coi_src_rate_err:.3f}"
                    row[f'NSIGMA']                  = f'{nsigma:.2f}'
                    #print(row)
                    
        
        t = Time(mjd, format='mjd')
        date = t.to_value('iso', subfmt='date')
        #row['simbad_name'] = simbad_name
        #row['local_name']  = local_name
        row['readable_name'] = readable_name
        row['ObsID'] = obsid
        row['Date']  = date
        
        row['rate_pc']   = f"{rate_pc:.2f} $\pm$ {rate_pc_err:.3f}"
        row['rate_soft'] = f"{rate_soft:.2f} $\pm$ {rate_soft_err:.3f}"
        row['rate_hard'] = f"{rate_hard:.2f} $\pm$ {rate_hard_err:.3f}"
        row['hr']        = f"{rate_hr:.2f} $\pm$ {rate_hr_err:.3f}"
        
        #print(row)
        
        all_rows.append(row)

        


{'readable_name': 'ESO 243-49 HLX-1'}
{'readable_name': 'Holmberg IX X-1'}
{'readable_name': 'Holmberg II X-1'}
{'readable_name': 'M31 ULX-1'}
{'readable_name': 'M33 ULX-1'}
{'readable_name': 'M51 ULX-7'}
{'readable_name': 'M81 X-6'}
{'readable_name': 'M82 X-2'}
{'readable_name': 'NGC1042 ULX-1'}
{'readable_name': 'NGC1313 X-1'}
{'readable_name': 'NGC1313 X-2'}
{'readable_name': 'NGC247 ULX-1'}
{'readable_name': 'NGC253 X-2'}
{'readable_name': 'NGC253 X-9'}
{'readable_name': 'NGC300 ULX-1'}
{'readable_name': 'NGC4395 ULX-1'}
{'readable_name': 'NGC5204 ULX-1'}
{'readable_name': 'NGC5408 ULX-1'}
{'readable_name': 'NGC55 ULX'}
{'readable_name': 'NGC5585 ULX'}
{'readable_name': 'NGC5907 ULX'}
{'readable_name': 'NGC6946 ULX-1'}
{'readable_name': 'NGC6946 ULX-3'}
{'readable_name': 'NGC7090 ULX-3'}
{'readable_name': 'NGC7793 P13'}
{'readable_name': 'NGC925 ULX-1'}
{'readable_name': 'NGC925 ULX-2'}
{'readable_name': 'SMC X-3'}
{'readable_name': 'SS433'}
{'readable_name': 'Swift J0243.6+6124'}


In [7]:
df = pd.DataFrame(all_rows)
savepath = '../tables/observation_table.tex'
print(f'Saving to {savepath}')
df.to_latex(savepath, longtable=False, escape=False)
print(df)

Saving to ../tables/observation_table.tex
         readable_name
0     ESO 243-49 HLX-1
1      Holmberg IX X-1
2      Holmberg II X-1
3            M31 ULX-1
4            M33 ULX-1
5            M51 ULX-7
6              M81 X-6
7              M82 X-2
8        NGC1042 ULX-1
9          NGC1313 X-1
10         NGC1313 X-2
11        NGC247 ULX-1
12          NGC253 X-2
13          NGC253 X-9
14        NGC300 ULX-1
15       NGC4395 ULX-1
16       NGC5204 ULX-1
17       NGC5408 ULX-1
18           NGC55 ULX
19         NGC5585 ULX
20         NGC5907 ULX
21       NGC6946 ULX-1
22       NGC6946 ULX-3
23       NGC7090 ULX-3
24         NGC7793 P13
25        NGC925 ULX-1
26        NGC925 ULX-2
27             SMC X-3
28               SS433
29  Swift J0243.6+6124
30         UGC6456 ULX
31             V404Cyg
32           M83 ULX-2
33           M83 ULX-1
34         NGC1365 X-1
35         NGC1365 X-2
36            IC10 X-1
37       NGC4945 XMM-1
38       NGC4559 ULX-1
39          M101 ULX-1


/tmp/ipykernel_545/1474920516.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(savepath, longtable=False, escape=False)


In [8]:
tab_uvot

NameError: name 'tab_uvot' is not defined

In [ ]:
from astropy.time import Time
t = Time(54060.8419942672, format='mjd')


In [ ]:
r